In [1]:
import torch
print(torch.backends.cudnn.enabled)
print(torch.version.cuda)
print(torch.cuda.is_available())

True
11.3
True


In [20]:
# import the relevant libraries
import numpy as np
import yaml
import cv2
from mss import mss
from PIL import Image

import torch
import torchvision

import ctypes
import time
import win32con
from utils.torch_utils import select_device
from utils.general import non_max_suppression
from utils.augmentations import letterbox

from models.common import DetectMultiBackend

#User parameters
show = True # boolean if the user wants to see the model output
control = False # boolean if the user wants the model to control the input device

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = select_device('') # a method which returns gpu availability, provided by YOLOv5

#LOAD_MODEL_FILE = "Halo640v2.pt" # the saved model file name
image_size = 1280 # size of the output screen
auto_screen = False # boolean should the program find the screen size automatically


centering = 1 #centering is the decimal percentage of the screen to account for, from the center, from 0 to 1

threshold = .8 # threshold is the lowest confidence bound to be allowed to be considered a detection

movement_scale =  640/image_size # a scaling factor based on the image size, required for proper mouse control.

with open('data/coco128.yaml', 'r', encoding="utf-8") as file:
    setup = yaml.safe_load(file)

#get the list of class names
class_list = setup['names']

YOLOv5  2022-10-2 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)



In [10]:
#model = torch.hub.load('ultralytics/yolov5', 'custom', path=LOAD_MODEL_FILE) # load the model trained on our Halo dataset
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval() # set the model to evaluation mode

Using cache found in C:\Users\blain/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-2 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 


requirements: C:\Users\blain\Documents\Git\AutoAim\YOLOv5\yolov5\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
           

In [ ]:
# if the user wants to
if auto_screen:
    w, h = ctypes.windll.user32.GetSystemMetrics(0), ctypes.windll.user32.GetSystemMetrics(1)
else:
    w, h = 1280,807 # if the user wants to define the screen size, do so here

# find the center point of the screen
x_center = w/2
y_center = h/2

# because we want to be able to take screen captures that do not use the entire screen, we need to define the top, bottom, left, and right bounds of the box in pixels
top = int(y_center - (y_center * centering))
left = int(x_center - (x_center * centering))
bottom = int(y_center + (y_center * centering))
right = int(x_center + (x_center * centering))

width = int(w * centering)
height = int(h * centering)

# we save these as a dictionary to be used with
monitor = {'top': top, 'left': left, 'width': width, 'height': height}

sct = mss() #instantiate the screenshot application

In [113]:
orig_img = sct.grab(monitor) # capture the image from the screen defined in the monitor dict
img = Image.frombytes('RGB', (orig_img.size.width, orig_img.size.height), orig_img.rgb) # perform several conversions to prepare the image for the model

with torch.no_grad():
    pred = model(img, size = 640).pred
print(pred[0])

img= np.array(img.convert('RGB'))

for box in pred[0]:
    class_num = int(box[5])
    cv2.rectangle(img,
                (int(box[0]), int(box[1])),
                (int(box[2]), int(box[3])),
                color=(0, 255, 0),
                thickness=2,
                )
    cv2.putText(img,
                org=(int(box[0]), int(box[1])), 
                text=str(class_list[class_num]), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=1, 
                color=(255,0,0),
                thickness=2
                )

        
cv2.imshow('Model View', np.array(img)[:,:,[2,1,0]])
if cv2.waitKey(1) & 0xff ==ord('q'): # The wait insures the image is shown, and the other portion of the command allows the user to stop the program by pressing 'q'
    cv2.destroyAllWindows()



tensor([[1.30863e+02, 4.76986e+02, 2.77494e+02, 8.07000e+02, 9.26919e-01, 0.00000e+00],
        [3.00258e+02, 4.82095e+02, 4.20447e+02, 8.02014e+02, 8.79163e-01, 0.00000e+00],
        [1.03029e+02, 3.08934e+02, 5.48144e+02, 8.00466e+02, 6.34115e-01, 5.00000e+00],
        [6.20172e+02, 6.65836e+02, 1.27924e+03, 8.07000e+02, 3.42174e-01, 7.00000e+00],
        [6.30769e+02, 6.63725e+02, 1.24961e+03, 8.07000e+02, 3.02772e-01, 5.00000e+00],
        [8.10389e+02, 7.45611e+02, 8.93487e+02, 8.07000e+02, 2.70259e-01, 0.00000e+00]], device='cuda:0')


In [114]:

# Begin the loop of screen inference
while True:
    start = time.time() # begin a timer to calculate framerate

    orig_img = sct.grab(monitor) # capture the image from the screen defined in the monitor dict
    img = Image.frombytes('RGB', (orig_img.size.width, orig_img.size.height), orig_img.rgb) # perform several conversions to prepare the image for the model

    with torch.no_grad():
        pred = model(img, size = 640).pred # perform inference of the image.  The

    img = np.array(img.convert('RGB')) # convert the image back to the correct color space and

    framerate = 1/(time.time() - start) # stop the timer and calculate the framerate

    cv2.putText(img, str(framerate), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA) #add framerate to the cv2 image

    if len(pred) > 0: # if there were any predicted targets
        closest = 99999 # set an arbitrarily large value for distance to the center
        for box in pred[0]: # loop through all predicted targets
        

            if box[4] > threshold: # if the prediction is above the user set confidence threshold
                box_dim = (box[3] - box[1])  #Caclulate the box dimension to be used to scale inputs later on
                x = (box[2] + box[0]) / 2 # find the x pixel value for the target
                y = ((box[3] + box[1]) / 2) - ((box[3] - box[1]) / 3) # find the y pixel value for the target (scale up by 1/3 to aim for headshots)
                class_num = int(box[5]) # find the class value for the target
                # getting the xywh bounding box coordinates
                # x, y, w, h = float(box[1]), float(box[2]), float(box[3]), float(box[4])
                        
                # add the target to the image
                cv2.circle(img,
                    (int(x), int(y)),
                    radius=10,
                    color=(255, 0, 0),
                    thickness=5
                )
                cv2.rectangle(img,
                            (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])),
                            color=(0, 255, 0),
                            thickness=2,
                            )
                cv2.putText(img,
                            org=(int(box[0]), int(box[1])), 
                            text=str(class_list[class_num]), 
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                            fontScale=1, 
                            color=(255,0,0),
                            thickness=2
                            )


                #get the coordinates in the full size space
                x = x / img.shape[1] * orig_img.size[0]
                y = y / img.shape[0] * orig_img.size[1]

                # Calculate the distance to the target
                dist_x = x.item()-x_center
                dist_y = y.item()-y_center
                dist = (dist_x**2 + dist_y**2)**(.5)

                # if this target is closer than the previous closest target, save its infromation as the closest target
                if dist < closest:
                    closest = dist
                    dx = dist_x
                    dy = dist_y
                    best_scale = box_dim/orig_img.size.height

        # if the user wants the program to control the mouse
        if control and closest < 99999:
            win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, # use the windows api to move the mouse
                                 int(.5*framerate* movement_scale*best_scale*dx), # this is the x increment to move, note the modifiers used to account for target distnace and framerate
                                 int(.5*framerate* movement_scale*best_scale*dy), # this is the y incrememnt to move, see above
                                 )

    if show:
        cv2.imshow('Model View', np.array(img)[:,:,[2,1,0]]) # if the user wants to see the model view, rearrange the channels and show the image

        if cv2.waitKey(1) & 0xFF == ord('q'): # The wait insures the image is shown, and the other portion of the command allows the user to stop the program by pressing 'q'
            cv2.destroyAllWindows() # destroy the image window
            break # step out of the loop
